# De-Americanization Algorithm
------------------
> **Idriss Afra**

This project aims to construct the dividends and forwards curves for American stocks and indices.

## US Stock & Index Options Market
Market-listed options on U.S. stocks and indices are typically American options, meaning they can be exercised on any business day prior to maturity. Since the call-put parity does not hold for American options, practitioners cannot use it to infer forward and dividend curves. As a result, various de-Americanization algorithms have been developed to address this limitation, with the simplest and most widely used approach being the following (For each listed expiry) :

Init :
- $n_{max}$, the maximum number of iterations
-   $F_0 = F(d_0)$, $d_0$ an initial dividend value guess and $F_0$ the initial forward value guess
- $K$, the strike of market call and put American options with the highest trading volume

While $n < n_{max}$ :
- Compute the implied American volatilities : $σ_{c}^{Amr}(K, F(d_k))$, $σ_{p}^{Amr}(K, F(d_k))$
- Compute the equivalent European prices : $C_{Eur}(K, σ_{c}^{Amr}(K, F(d_k)))$, $P_{Eur}(K, σ_{p}^{Amr}(K, F(d_k)))$
- Compute the forward using the call-put parity :

$$F_{k+1} = e^{rT} \left(C_{Eur}(K, σ_{c}^{Amr}(K, F(d_k))) - P_{Eur}(K, σ_{p}^{Amr}(K, F(d_k)))\right) + K $$

- Imply the associated dividend yield : $d_{k+1} = ln\left(\frac{Spot × e^{rT}}{F_{k+1}}\right) \div T$
- If :  $|F(d_k+1) - F(d_{k})| > ϵ$, Continue. Else : Break.

In this project, we use the Binomial model to imply American volatilities and price equivalent European options.

## The Binomial Model

The Binomial model is relatively easy to understand and implement as it assumes that the underlying asset $S_t$ moves up or down by a fixed percentage in each period.

Let's define the movement factors $u$ and $d$ as following :
$$
u = e^{\sigma\sqrt{\Delta t}}\qquad d = e^{-\sigma\sqrt{\Delta t}} \\
$$
Where $\sigma$ is the volatility and $Δt$ is a tiny time step to maturity $T$. The risk-neutral probability under this model is :

$$
p = \frac{e^{(r-q)\Delta t}-d}{u-d} \\
$$
So that :
$$
E(S_{t+\Delta t}) = p × u × S_t + (1 - p) × d × S_t  
= p × S_{t+\Delta t}^u + (1 - p) × S_{t+\Delta t}^d \\
$$

Where $r$ is the zero-coupon interest rate and $q$ the dividend yield.

Once the Binomial tree simulation is done, the options are priced using a backward method :

* European Style :  
$$
PV_t(S_t) = e^{-r\Delta t}\left(p\times PV_{t+\Delta t}(S_{t+\Delta t}^u)+(1-p)\times PV_{t+\Delta t}(S_{t+\Delta t}^d)\right)
$$

* American Style :    
$$
PV_t(S_t) = max\left(S_t - K, e^{-r\Delta t}\left(p\times PV_{t+\Delta t}(S_{t+\Delta t}^u)+(1-p)\times PV_{t+\Delta t}(S_{t+\Delta t}^d)\right)\right) \\
$$

With the final condition : $ PV_T(S) = Max\left(ϕ × (S - K), 0 \right)  $

Where $ϕ = 1$ for call options and $ϕ = -1$ for put options, and $K$ is the strike price.

The accuracy of the binomial model improves with the number of time steps. However, this also raises the complexity and computation time.

In [1]:
import numpy as np
import math

def spot_simulation(S0, r, sigma, T, n_steps):
    """
    The spot simulation under the Binomial model.
    The function returns the Binomial tree.
    """
    dt = T / n_steps
    u = math.exp(sigma * math.sqrt(dt))
    d = 1. / u

    s = np.zeros((n_steps+1, n_steps+1))
    s[0,0] = S0
    for i in range(1, n_steps+1) :
        s[:i,i] = s[:i,i-1] * u
        s[i,i] = s[i-1,i-1] * d

    return s

# Test
spot_simulation(100, 0.03, 0.2, 1, 3)

array([[100.        , 112.24009024, 125.97837858, 141.39824581],
       [  0.        ,  89.09472523, 100.        , 112.24009024],
       [  0.        ,   0.        ,  79.37870064,  89.09472523],
       [  0.        ,   0.        ,   0.        ,  70.72223522]])

In [2]:
def tree_price(flavor, K, T, S0, r, sigma, q, amer=True, n_steps=750):
    """
    The Binomial model.
    The function returns the Binomial price.
    """
    # Spot simulation
    dt = T / n_steps
    u = math.exp(sigma * math.sqrt(dt))
    d = 1. / u
    p = (math.exp((r - q) * dt) - d) / (u - d)
    s = np.zeros((n_steps+1, n_steps+1))
    s[0,0] = S0
    for i in range(1, n_steps+1) :
        s[:i,i] = s[:i,i-1] * u
        s[i,i] = s[i-1,i-1] * d

    # Option payoff
    phi = 1 if flavor.upper() == "CALL" else -1
    v = np.maximum(phi * (s[:, n_steps] - K), 0.) # payoff at final date T

    # Discount between 2 time steps
    discount = math.exp(-r * dt)

    # Backward loop
    for i in range(n_steps-1, -1, -1) : # => i = n_steps-1 ... 0
        n_nodes = i+1 # i+1 nodes at time step #i
        v = discount * ( p * v[:n_nodes] + (1-p) * v[1:n_nodes+1] )
        if amer :
            v = np.maximum(phi * (s[:n_nodes, i] - K), v)

    return v[0]

# Test
print("Example of a Put option on stock with dividends :")
print("-------------------------------------------------")
print("American Put price", round(tree_price("Put", 100, 1, 105, 0.03, 0.2, 0.01), 4))
print("European Put price", round(tree_price("Put", 100, 1, 105, 0.03, 0.2, 0.01, False), 4))

Example of a Put option on stock with dividends :
-------------------------------------------------
American Put price 5.1462
European Put price 5.0191


## Implied American Volatilities

The first step in the de-Americanization algorithm involves calibrating American volatilities by using market-listed call and put options with the highest trading volumes. For this step, we employ the Bisection method:

In [3]:
import sys

def amer_implied_vol(S0, r, T, K, flavor, q, market_price) :
    """
    Bisection algorithm on the Binomial model.
    The function returns the implied American vol.
    """
    sigma_min = 0.001
    sigma_max = 10
    n_max = 750
    eps = 1e-5

    n = 1
    while n <= n_max :
        sigma = (sigma_min + sigma_max)  / 2
        f_sigma = tree_price(flavor, K, T, S0, r, sigma, q) - market_price
        if (f_sigma == 0) or (sigma_max - sigma_min < eps) :
            return sigma
        if f_sigma < 0 :
            sigma_min = sigma
        else :
            sigma_max = sigma
        n += 1

    return sys.exit("The American volatility calibration algorithm failed to converge. Please review data of the " + flavor.upper() + " with Strike " + str(K))

# Test
print(str(100 * round(amer_implied_vol(100, 0.03, 1, 105, "Call", 0.01, 6.64), 4)) + "%")

20.0%


## De-Americanization Algorithm

In [4]:
def de_Americanization(T, S0, r, q_init, K, flavors, amer_prices) :
    """
    The de-Americanization algorithm function :
    1- Call the amer_implied_vol function to calibrate volatilities of the American call and put options with the highest trading volume (American Binomial model)
    2- Use these volatilities to re-price the equivalent European call and put prices (European Binomial model)
    3- Imply the forward using the call-put parity
    4- Re-do the 1st, 2nd, and 3rd steps until the convergence of the implied forward
    """
    n_max = 750
    eps = 1e-3

    if flavors[0].upper() == "CALL" :
        sigma_call = amer_implied_vol(S0, r, T, K, flavors[0], q_init, amer_prices[0])
        sigma_put = amer_implied_vol(S0, r, T, K, flavors[1], q_init, amer_prices[1])
    else :
        sigma_call = amer_implied_vol(S0, r, T, K, flavors[1], q_init, amer_prices[1])
        sigma_put = amer_implied_vol(S0, r, T, K, flavors[0], q_init, amer_prices[0])

    eur_price_call = tree_price("Call", K, T, S0, r, sigma_call, q_init, False)
    eur_price_put = tree_price("Put", K, T, S0, r, sigma_put, q_init, False)

    F = math.exp(r * T) * (eur_price_call - eur_price_put) + K
    q = math.log(S0 / F) / T + r
    nb_iteration = 1

    print("-----------------------------------------------------")
    print("Step : ", nb_iteration)
    print("Implied Forward : ", round(F, 4))
    print("Implied Dividend Yield : " + str(round(100 * q, 4)) + "%")

    while nb_iteration < n_max :
        if flavors[0].upper() == "CALL" :
            sigma_call = amer_implied_vol(S0, r, T, K, flavors[0], q, amer_prices[0])
            sigma_put = amer_implied_vol(S0, r, T, K, flavors[1], q, amer_prices[1])
        else :
            sigma_call = amer_implied_vol(S0, r, T, K, flavors[1], q, amer_prices[1])
            sigma_put = amer_implied_vol(S0, r, T, K, flavors[0], q, amer_prices[0])

        print("Market American Call Price : ", round(amer_prices[0], 4))
        print("Binomial American Call Price : ", round(tree_price(flavors[0], K, T, S0, r, sigma_call, q), 4))
        print("Implied American Call Vol : " + str(round(100 * sigma_call, 4)) + "%")
        print("Market American Put Price : ", round(amer_prices[1], 4))
        print("Binomial American Put Price : ", round(tree_price(flavors[1], K, T, S0, r, sigma_put, q), 4))
        print("Implied American Put Vol : " + str(round(100 * sigma_put, 4)) + "%")

        eur_price_call = tree_price("Call", K, T, S0, r, sigma_call, q, False)
        eur_price_put = tree_price("Put", K, T, S0, r, sigma_put, q, False)

        print("Equivalent Binomial European Call Price : ", round(eur_price_call, 4))
        print("Equivalent Binomial European Put Price : ", round(eur_price_put, 4))

        old_F = F
        F = math.exp(r * T) * (eur_price_call - eur_price_put) + K
        q = math.log(S0 / F) / T + r

        if abs(F - old_F) < eps :
            return F

        nb_iteration += 1
        
        print("-----------------------------------------------------")
        print("Step : ", nb_iteration)
        print("Implied Forward : ", round(F, 4))
        print("Implied Dividend Yield: " + str(round(100 * q, 4)) + "%")

    return sys.exit("The Dividends calibration algorithm failed to converge. Please review the inputs")


## Application : AAPL US Listed Equity Options

Let's implement the de-Americanization algorithm outlined above on the Apple US stock options as of August 10, 2023:

In [5]:
from datetime import date
import matplotlib.pyplot as plt

now_date = date(2023, 8, 10)
S0 = 177.83
r = 5.482 / 100

exp = date(2023, 12, 15)
T = (exp - now_date).days / 365
# OTM listed American calls and puts prices
strikes = [165, 170, 175, 180, 185, 190]
amer_prices = [4.425, 5.75, 7.425, 10.125, 7.625, 5.5250]
flavors = ["Put"] * 3 + ["Call"] * 3

# The highest trading volume strike asof 8 Aug 2023 is 185. Its associated call option close price is 7.625 and its put option close price is 11.975
q_init = 0.0001
print("Dividend Yield Initial Guess : " + str(round(100 * q_init, 4)) + "%")
fwd = de_Americanization(T, S0, r, q_init, 185, ["Call", "Put"], [7.625, 11.975])
div_yield = math.log(S0 / fwd) / T + r
div_cash = S0 * math.exp(r * T) - fwd

print("-----------------------------------------------------")
print("Implied Fwd : ", round(fwd, 4))
print("Implied Dividend Yield : " + str(round(100 * div_yield, 4)) + "%")
print("Implied Cash Dividend : ", round(div_cash, 4))

Dividend Yield Initial Guess : 0.01%
-----------------------------------------------------
Step :  1
Implied Forward :  181.1041
Implied Dividend Yield : 0.2387%
Market American Call Price :  7.625
Binomial American Call Price :  7.625
Implied American Call Vol : 22.2912%
Market American Put Price :  11.975
Binomial American Put Price :  11.975
Implied American Put Vol : 22.3437%
Equivalent Binomial European Call Price :  7.625
Equivalent Binomial European Put Price :  11.469
-----------------------------------------------------
Step :  2
Implied Forward :  181.082
Implied Dividend Yield: 0.2737%
Market American Call Price :  7.625
Binomial American Call Price :  7.6248
Implied American Call Vol : 22.3151%
Market American Put Price :  11.975
Binomial American Put Price :  11.9751
Implied American Put Vol : 22.3236%
Equivalent Binomial European Call Price :  7.6248
Equivalent Binomial European Put Price :  11.4723
-----------------------------------------------------
Step :  3
Implied F

## Dividend Seasonality : Interpolation & Extrapolation

US companies typically pay cash dividends on a quarterly, semi-annual, or annual basis, with a noticeable seasonality in their dividend patterns. As a result, practitioners often forecast future dividend payment dates by assuming the same payment days and months as in previous years. A new "seasoned" dividend curve is then bootstrapped as follows:

Under the Non-Arbitrage assumption : $F_{T^{Exp}_n} = S_t × e^{r (T^{Exp}_n-t)} - ∑_{k ∈ [1,n]}d_k × e^{r(T^{Exp}_n-T_k)}$ 

Therefore :  $d_n = e^{-r\left(T^{Exp}_n-T_n\right)} × \left( S_t × e^{r \left(T^{Exp}_n-t\right)} - ∑_{k ∈ [1,n-1]}d_k × e^{r\left(T^{Exp}_n-T_k\right)} - F_{T^{Exp}_n} \right)$ 

And finally : $d_n = S_t × e^{r \left(T_n-t\right)} - ∑_{k ∈ [1,n-1]}d_k × e^{r\left(T_n-T_k\right)} - F_T × e^{-r(T-T_n)}$ 


Where  :

* $S_t$ the spot price at $t$
* $r$ the zero-coupon rate
* $\left(F_{T^{Exp}_n}\right)_n$ are the implied forwards at the market option expiries $\left(T^{Exp}_n\right)_n$
* $(d_k)_k$ are the projected cash dividends paid at the projected Ex-dividend dates $(T_k)_k$

On the other hand, practitioners typically assume continuous dividend yield curves for the indices. The most straightforward method to construct these curves is by assuming a piecewise constant curve between the market option expiries:

Under the Non-Arbitrage assumption : $F_{T^{Exp}_n} = S_t × e^{r \left(T^{Exp}_n-t\right) - ∑_{k ∈ [1,n]}q_k × \left(T^{Exp}_{k} - T^{Exp}_{k-1} \right)}$ 

Therefore : $q_n = ln\left( S_t × e^{r \left(T^{Exp}_n-t\right) - ∑_{k ∈ [1,n-1]}q_k × \left(T^{Exp}_{k} - T^{Exp}_{k-1} \right)} / F_{T^{Exp}_n}\right) / \left( T^{Exp}_n - T^{Exp}_{n-1}\right)$

Where  : $(q_k)_k$ are the piecewise constant instantaneous forward dividend yields between $\left( T^{Exp}_k - T^{Exp}_{k-1}\right)$

Cash and yield dividends can then be flat extrapolated after the last market option expiry.

## Implied European Volatilities

Once the forward and dividend curves are calibrated, the next step is to compute the implied European volatilities to price European derivatives on a U.S. underlying stock. To achieve this, we assume that the equivalent European Binomial prices are the market prices and use these to derive the implied European volatilities under the Black-Scholes standard market model (For each listed expiry):

Init :
- $F = F(d)$, $d$ and $F$ are the calibrated dividend yield and forward price by the above de-Americanization algorithm
- $Strikes$, the quoted strikes of OTM American call and put options

For $K$ in $Strikes$ :
- Compute the implied American Binomial volatility : $σ_{Amr}(K, F(d))$
- Compute the equivalent European Binomial price : $Binomial_{EUR}(K, σ_{Amr}(K, F(d)))$
- Calibrate the Implied European Volatilities $\sigma_{Imp}(K,T)$ such that :

$$
BS(F(d), K, \sigma_{Imp}(K,T), T) - Binomial_{EUR}(K, σ_{Amer}(K, F(d))) = 0  $$

The $\sigma_{Imp}(K,T)$ calibration is done via the Newton-Raphson method.



In [6]:
from scipy.stats import norm
from scipy import optimize

def bs_price(flavor, K, T, S0, r, vol, q=0) :
    """
    The Black-Scholes price.
    """
    v2T = vol**2 * T
    d1 = (np.log(S0/K) + (r - q) * T + v2T / 2) / v2T**0.5
    d2 = d1 - v2T**0.5
    phi = 1 if flavor.upper() == "CALL" else  -1
    return phi * (S0 * math.exp(-q * T) * norm.cdf(phi * d1) - K * math.exp(-r * T) * norm.cdf(phi * d2))

def bs_vega(K, T, S0, r, vol, q=0) :
    """
    The Black-Scholes Vega greek : The derivative of the option value with respect to the volatility of the underlying asset.
    """
    v2T = vol**2 * T
    d2 = (math.log(S0/K) + (r - q) * T - v2T / 2) / v2T**0.5
    return K * math.exp(-r * T) * norm.pdf(d2) * T**0.5

def newton_raphson(market_price, init_vol, flavor, K, T, S0, r, q=0):
    """
    The Newton-Raphson algorithm : Implied volatilities from market prices
    """
    f = lambda vol : bs_price(flavor, K, T, S0, r, vol, q) - market_price
    f_deriv = lambda vol : bs_vega(K, T, S0, r, vol, q)
    return optimize.newton(f, init_vol, f_deriv, maxiter=250, tol=1e-06)
    
def euro_equivalent_vol(T, S0, r, q, K, flavor, amer_price) :
    """
    This function calibrates the implied European volatility of a given strike K and expiry T
    """
    amr_vol = amer_implied_vol(S0, r, T, K, flavor, q, amer_price)
    eur_price = tree_price(flavor, K, T, S0, r, amr_vol, q, False)
    return newton_raphson(eur_price, amr_vol, flavor, K, T, S0, r, q)

Let us get back our example of AAPL US stock options and calibrate its equivalent implied European volatilities :

In [7]:
for i in range(len(strikes)) :
    amr_vol = 100 * amer_implied_vol(S0, r, T, strikes[i], flavors[i], div_yield, amer_prices[i])
    eur_vol = 100 * euro_equivalent_vol(T, S0, r, div_yield, strikes[i], flavors[i], amer_prices[i])
    print("American implied vol at " + str(strikes[i]) + " : " + str(round(amr_vol, 4)) + "%")
    print("European implied vol at " + str(strikes[i]) + " : " + str(round(eur_vol, 4)) + "%")
    print("Difference in Bps : ", round(10000 * abs(amr_vol - eur_vol), 2))

American implied vol at 165 : 25.9453%
European implied vol at 165 : 25.9508%
Difference in Bps :  54.77
American implied vol at 170 : 24.9803%
European implied vol at 170 : 24.9817%
Difference in Bps :  14.4
American implied vol at 175 : 24.0763%
European implied vol at 175 : 24.0841%
Difference in Bps :  77.94
American implied vol at 180 : 23.014%
European implied vol at 180 : 23.0168%
Difference in Bps :  27.53
American implied vol at 185 : 22.3198%
European implied vol at 185 : 22.3177%
Difference in Bps :  21.6
American implied vol at 190 : 21.6151%
European implied vol at 190 : 21.6123%
Difference in Bps :  27.96
